In [1]:
# Import library
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

import xgboost as xgb
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import time
from tqdm import tqdm_notebook as tqdm
from IPython.display import IFrame

# Ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

# Set the seed for randomization
seed = 0

In [2]:
def download_file(url):
    '''
    Download file from given URL
    
    Parameters
    ----------
    url: str
        URL of the file to download
    '''
    
    # One chunk = 1KB = 1024 byte
    chunk_size = 1024
    # Create a request object to download the file in streaming mode
    r = requests.get(url, stream = True)
    # Get the total size of the file
    total_size = int(r.headers['content-length'])
    # Use the original filename (The string after the last "/")
    filename = url.split('/')[-1]
    
    # Open a new file with filename and write the download data
    with open(filename, 'wb') as f:
        # Use tqdm to show download progress
        for data in tqdm(iterable = r.iter_content(chunk_size = chunk_size), 
                         total = total_size/chunk_size, unit ='KB'):
            # Write the data to file
            f.write(data)
            
    print('Donwload complete')

In [3]:
# The URL of the Green Taxis data from September 2015
url = "https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv"

In [5]:
# Download the file
download_file(url)

Donwload complete


In [12]:
# Load the csv file into pandas dataframe
data = pd.read_csv('data/yellow_tripdata_2018-01.csv')

In [19]:
# Convert data type for categorical attribute
data.VendorID = data.VendorID.astype(object)
data.RatecodeID = data.RatecodeID.astype(object)
data.PULocationID = data.PULocationID.astype(object)
data.DOLocationID = data.DOLocationID.astype(object)
data.payment_type = data.payment_type.astype(object)
# Convert data type for date attribute
data.tpep_pickup_datetime = pd.to_datetime(data.tpep_pickup_datetime)
data.tpep_dropoff_datetime = pd.to_datetime(data.tpep_dropoff_datetime)

In [20]:
data.dtypes

VendorID                         object
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                       object
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                     object
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

In [21]:
data.shape

(8759874, 17)